#### autoreload modules and utilities

In [5]:
%reload_ext autoreload
%autoreload 2

#### import all neceesary libraries/packages

In [9]:
import joblib

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

from wordcloud import WordCloud

from sklearn.pipeline import Pipeline 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

#### Import necessary utilities

The utilities file is for cleaning of the text

In [10]:
## utilities
from utils import clean_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DHREY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DHREY\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DHREY\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load all the data

We will be using combination of of imdb, yelp and amazon cells users' review

In [ ]:
df_imdb = pd.read_csv('data/imdb_labelled.txt', sep='\t', names=[ 'review', 'label'])

df_yelp = pd.read_csv('data/yelp_labelled.txt', sep='\t', names=[ 'review', 'label'])

df_amzn = pd.read_csv('data/amazon_cells_labelled.txt', sep='\t', names=[ 'review', 'label'])

Checking the shape for each dataset

In [ ]:
df_imdb.shape, df_yelp.shape, df_amzn.shape

###### We join the dataset together

In [ ]:
data = df_imdb.append([df_yelp, df_amzn], ignore_index=True)

In [ ]:
# Checeking the data shape
data.shape

### Create Label (Positive/Negative)

In [ ]:
data['sentiment'] = data["label"].apply(lambda x: "positive" if x else "negative")
data.head()

### Missing values

In [ ]:
data.isnull().sum()

###### We check for the sum of missing values if there exist any, fortunately there is no missing value in the dataset 

### Label Frequency

In [ ]:
Index = [1, 0]

print(data["sentiment"].value_counts())
print()

barlist = plt.bar(Index, data["sentiment"].value_counts())

plt.title("Frequency of Sentiments")
plt.xticks(Index, ['positive', 'negative'])
plt.ylabel('Number of Reviews')
plt.xlabel('Sentiment expressed in Reviews')

barlist[Index[1]].set_color('green')
barlist[Index[0]].set_color('red')
plt.show()

The Dataset labels seems to be balanced, there will be no need for any under/over sampling

### Cleaning of Data
This is done by using utilities function(clean_text) as .apply parameter

In [ ]:
data['clean_review'] = data["review"].apply(clean_text)
data.head()

In [ ]:
data['clean_review'].head()

# Visualisation of Review

We will be visualising the review using wordcloud module. The module visualised the most frequent words in large size and the less frequent words in smaller sizes.

#### Objective:
    1. To know the most frequent words
    2. Words that contributed to Negative stress
    3. Words that contributed to Positive stress

In [ ]:
words = ''.join([text for text in data['clean_review']])

cloud_word = WordCloud(width= 600, height = 500, random_state = 31, max_font_size = 100, background_color='white').generate(words)

plt.figure(figsize = (10, 15))
plt.imshow(cloud_word, interpolation = "bilinear")
plt.axis('off')
plt.show()

We can see most of the words are positive or neutral. With film, movie, good, character being the most frequent ones.  Similarly, we will plot the word cloud for the Positive and Negative score.

In [ ]:
positive_words = ''.join([text for text in data['clean_review'][data['label']==1]])

cloud_word = WordCloud(width= 600, height = 500, max_font_size = 100, background_color='green').generate(positive_words)

plt.figure(figsize = (10, 25))
plt.imshow(cloud_word, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
negative_words = ''.join([text for text in data['clean_review'][data['label']==0]])

cloud_word = WordCloud(width= 600, height = 500, max_font_size = 100, background_color='black').generate(negative_words)

plt.figure(figsize = (10, 15))
plt.imshow(cloud_word, interpolation = "bilinear")
plt.axis('off')
plt.show()

Despite the fact that it was saturated by common words like film, watch, very etc; there are nagative words like bore, suck, redicule etc.

## Text preprocessing

#### Observe count vectorizer

In [ ]:
count_vectorizer = CountVectorizer(ngram_range=(1,1), stop_words='english')
count_data = count_vectorizer.fit_transform(data["clean_review"])
cv_dataframe = pd.DataFrame(count_data.toarray(), columns=count_vectorizer.get_feature_names())

cv_dataframe.head()

#### Observe TFIDF vectorizer

In [ ]:
tfidf_vect = TfidfVectorizer(ngram_range=(1,1),stop_words='english')

tfidf_data = tfidf_vect.fit_transform(data["clean_review"])
tfidf_dataframe = pd.DataFrame(tfidf_data.toarray(), columns= tfidf_vect.get_feature_names())

tfidf_dataframe.head()

###### What CountVectorizer and Vectorizers do is to split the sentences into words, count the number of occurences of each word which is called bag of word model, the higher the frequency of a word really means its not so important or close to irrelevance, therefore the inverse logarithm is been carried out, with the result, you can easily deduce that the closer the value is towards 0 the less relevant it is. They are a very useful preprocessing tool used for text classification model.

In [ ]:
X = data['clean_review']
y = data['label']

###### This is where we split the dataset in preparation for the training using a sklearn module named train_test_split, reviews ia what we are using to make predictions therefore it is our feature in this case and scores of the reviews is our target variable, what we are predicting.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

print(f"train shape input:{X_train.shape}, output:{y_train.shape}")
print(f"test shape  input:{X_test.shape}, output:{y_test.shape}")

## Train with TfidfVector and Multinomial NB


#### initilize pipeline with TfidfVector

In [ ]:
pipeNB = Pipeline([
    ('bow',TfidfVectorizer(stop_words="english")), 
    ('classifier',MultinomialNB())
])

In [ ]:
pipeNB.fit(X_train, y_train)

#### Creating joblib for deploment

In [ ]:
joblib.dump(pipeNB, "models/multinomial_naive_bayes_with_tfidf_vectorizer.joblib")

In [ ]:
y_pred = pipeNB.predict(X_test) #predict testing data

print(classification_report(y_test, y_pred))

### Running Cross validation with TfidfvVectorizer and MultinomialNB

In [ ]:
accuracy = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=100) 
skf.get_n_splits(X, y) 

for train_index, test_index in skf.split(X, y):
    x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index] 
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    pipeNB.fit(x_train_fold, y_train_fold)
    result = pipeNB.score(x_test_fold, y_test_fold)
    accuracy.append(result)

accuracy = np.array(accuracy)

# Print the output
print('List of first 10 possible accuracy:')
for index, acc in enumerate(accuracy[:10]):
    print(f"{index+1:3d}. {acc:.4f}")

print('\nMetrics that were obtained from this model:')
print(f' Maximum Accuracy:   {accuracy.max()*100:.2f}%') 
print(f' Minimum Accuracy:   {accuracy.min()*100:.2f}%') 
print(f' Mean Accuracy:      {accuracy.mean()*100:.2f}%') 
print(f' Standard Deviation: {accuracy.std():.4f}')

### Training Countvectorizer and MultinomialNB

In [ ]:
pipeNB = Pipeline([
    ('bow',CountVectorizer(stop_words="english")),
    ('classifier',MultinomialNB())
])

In [ ]:
pipeNB.fit(X_train, y_train)

#### Creating joblib for deploment

In [ ]:
joblib.dump(pipeNB, "models/multinomial_naive_bayes_with_count_vectorizer.joblib")

In [ ]:
y_pred = pipeNB.predict(X_test) #predict testing data

print(classification_report(y_test, y_pred))

### Running Cross validation with Countvectorizer and MultinomialNB

In [ ]:
accuracy = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=100) 
skf.get_n_splits(X,y) 

for train_index, test_index in skf.split(X, y):
  X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index] 
  y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

  pipeNB.fit(X_train_fold, y_train_fold)
  result = pipeNB.score(X_test_fold, y_test_fold)
  accuracy.append(result)


accuracy = np.array(accuracy)

# Print the output
print('List of first 10 possible accuracy:')
for index, acc in enumerate(accuracy[:10]):
    print(f"{index+1:3d}. {acc:.4f}")

print('\nMetrics that were obtained from this model:')
print(f' Maximum Accuracy:   {accuracy.max()*100:.2f}%') 
print(f' Minimum Accuracy:   {accuracy.min()*100:.2f}%') 
print(f' Mean Accuracy:      {accuracy.mean()*100:.2f}%') 
print(f' Standard Deviation: {accuracy.std():.4f}')